<a href="https://colab.research.google.com/github/IsaacFigNewton/Analyzing-Hate-Crime-Data/blob/main/Hate_Crime_Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Stuff

###Import all libraries

In [160]:
import pandas as pd
import sklearn as sk
import numpy as np
import seaborn as sb
import re

###Import datasets

In [161]:
crime_df = pd.read_csv("https://raw.githubusercontent.com/IsaacFigNewton/Analyzing-Hate-Crime-Data/main/hate_crime/hate_crime.csv", on_bad_lines='skip')
city_demo_df = pd.read_csv("https://raw.githubusercontent.com/IsaacFigNewton/Analyzing-Hate-Crime-Data/main/demographics/city/ACSST1Y2022.S0101-Data.csv", on_bad_lines='skip')
county_demo_df = pd.read_csv("https://raw.githubusercontent.com/IsaacFigNewton/Analyzing-Hate-Crime-Data/main/demographics/county/ACSDP1Y2022.DP05-Data.csv", on_bad_lines='skip')

#Data Cleaning

In [162]:
# use the entries of the first row as the column headers for easier management
def fixHeaders(df):
  new_headers = df.iloc[0]
  new_df = df[1:]
  new_df.columns = new_headers
  return new_df

In [163]:
#fix the headers
city_demo_df = fixHeaders(city_demo_df)
county_demo_df = fixHeaders(county_demo_df)

In [174]:
def split_area_name(area):
    result = [np.nan, np.nan, np.nan]
    if ", " in area:
        result = area.split(", ") + [np.nan]
    if " city" in result[0].lower():
        result[0] = result[0][0:-5]
        result[2] = result[1]
        result[1] = "City"
    elif " county" in result[0].lower():
        result[0] = result[0][0:-7]
        result[2] = result[1]
        result[1] = "County"

    return result + [np.nan] * (3 - len(result))

def splitArea(df):
    df[["pug_agency_name", "agency_type_name", "state_name"]] = df["Geographic Area Name"].map(split_area_name).apply(pd.Series)

In [176]:
splitArea(city_demo_df)
splitArea(county_demo_df)

city_demo_df.head()

,Geography,Geographic Area Name,Estimate!!Total!!Total population,Margin of Error!!Total!!Total population,Estimate!!Total!!Total population!!AGE!!Under 5 years,Margin of Error!!Total!!Total population!!AGE!!Under 5 years,Estimate!!Total!!Total population!!AGE!!5 to 9 years,Margin of Error!!Total!!Total population!!AGE!!5 to 9 years,Estimate!!Total!!Total population!!AGE!!10 to 14 years,Margin of Error!!Total!!Total population!!AGE!!10 to 14 years,...,Estimate!!Percent Female!!Total population!!SUMMARY INDICATORS!!Child dependency ratio,Margin of Error!!Percent Female!!Total population!!SUMMARY INDICATORS!!Child dependency ratio,Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex,Margin of Error!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex,Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!!Age,Margin of Error!!Percent Female!!Total population!!PERCENT ALLOCATED!!Age,NaN,pug_agency_name,agency_type_name,state_name
1,1600000US0103076,"Auburn city, Alabama",80009,20,3657,1311,3809,1134,4348,1136,...,(X),(X),(X),(X),(X),(X),NaN,Auburn,City,Alabama
2,1600000US0107000,"Birmingham city, Alabama",196353,868,11495,2020,11631,2173,9404,2476,...,(X),(X),(X),(X),(X),(X),NaN,Birmingham,City,Alabama
3,1600000US0121184,"Dothan city, Alabama",70524,678,4549,468,5059,662,4146,592,...,(X),(X),(X),(X),(X),(X),NaN,Dothan,City,Alabama
4,1600000US0135896,"Hoover city, Alabama",92427,45,5778,1406,5805,1516,7377,2051,...,(X),(X),(X),(X),(X),(X),NaN,Hoover,City,Alabama
5,1600000US0137000,"Huntsville city, Alabama",222363,2301,12180,1698,13883,1837,10494,1917,...,(X),(X),(X),(X),(X),(X),NaN,Huntsville,City,Alabama


In [177]:
#only consider the 2022 crime data
crime_df = crime_df[crime_df['data_year'] == 2022]

###Combine the datasets

In [201]:
#only include entries corresponding to actual incidents
# merged_df = crime_df.merge(city_demo_df, on=["pug_agency_name", "agency_type_name", "state_name"], how="left")
# merged_df = merged_df.merge(county_demo_df, on=["pug_agency_name", "agency_type_name", "state_name"], how="left")
# merged_df = merged_df[pd.notna(merged_df['incident_id'])]

# Merge city demographics with crime dataframe
merged_city_df = crime_df.merge(city_demo_df, on=["pug_agency_name", "agency_type_name", "state_name"], how="left")

# Merge county demographics with crime dataframe
merged_county_df = crime_df.merge(county_demo_df, on=["pug_agency_name", "agency_type_name", "state_name"], how="left")

# Concatenate both dataframes along the rows
merged_df = pd.concat([merged_city_df, merged_county_df])

# Remove rows where incident_id is NaN
merged_df = merged_df.dropna(subset=['incident_id'])

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [200]:
merged_df

,incident_id,data_year,ori,pug_agency_name,pub_agency_unit,agency_type_name,state_abbr,state_name,division_name,region_name,...,"Percent Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or More Races!!Two races excluding Some Other Race, and three or more races",Percent!!Total housing units,Percent Margin of Error!!Total housing units,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female",NaN
0,1476719,2022,AK0010500,Kodiak,NaN,City,AK,Alaska,Pacific,West,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1483026,2022,AK0015600,Wasilla,NaN,City,AK,Alaska,Pacific,West,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1484192,2022,AK0010100,Anchorage,NaN,City,AK,Alaska,Pacific,West,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1484193,2022,AK0010100,Anchorage,NaN,City,AK,Alaska,Pacific,West,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1484194,2022,AK0010100,Anchorage,NaN,City,AK,Alaska,Pacific,West,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11638,1473449,2022,WY0190200,Rock Springs,NaN,City,WY,Wyoming,Mountain,West,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11639,1473450,2022,WY0010200,University of Wyoming,NaN,University or College,WY,Wyoming,Mountain,West,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11640,1476554,2022,WY0190000,Sweetwater,NaN,County,WY,Wyoming,Mountain,West,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11641,1476555,2022,WY0110100,Cheyenne,NaN,City,WY,Wyoming,Mountain,West,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
